In [13]:
import asyncio
import requests
from datetime import datetime, timedelta
import pandas as pd
from scipy.integrate import trapezoid
import math
from statistics import mean
from io import StringIO
import logging

In [14]:
logging.basicConfig(format='%(levelname)-8s [%(filename)s:%(lineno)d] %(message)s',level=logging.DEBUG)

In [15]:
async def send_get_request(url:str='http://participant0.local:5000/',endpoint:str='',type:str='json',key=None,timeout=1):
        """Send GET request to the IP."""
        # get own data
        max_tries = 3

        if key:
            headers = {"Authorization": f"Bearer {key}"}
        else:
            headers = {}
            
        for attempt in range(max_tries):
            try:
                response = requests.get(f"{url}{endpoint}",headers=headers, timeout=timeout)
                response.raise_for_status()
                if type == 'json':
                    res= parse_datetimes(convert_bools(response.json()))
                elif type == 'text':
                    res= response.text
                else:
                    res= response.status_code
                break
            except requests.exceptions.HTTPError as e:
                logging.error(f"HTTP error occurred: {e}")
            except Exception as e:
                logging.error(f'{e}')
                if attempt == max_tries-1: # try up to 3 times
                    return None
                else:
                    logging.debug('SLEEEEEEEEEEEEEEEEEPING')
                    await asyncio.sleep(1+attempt)
        return res

# # convert datetimes to iso formatted strings
# def convert_datetimes(obj):
#     if isinstance(obj, dict):
#         return {k: convert_datetimes(v) for k, v in obj.items()}
#     elif isinstance(obj, list):
#         return [convert_datetimes(i) for i in obj]
#     elif isinstance(obj, datetime):
#         return obj.isoformat()
#     else:
#         return obj

# convert to datetimes from iso formatted strings
def parse_datetimes(obj):
    if isinstance(obj, dict):
        return {k: parse_datetimes(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [parse_datetimes(i) for i in obj]
    elif isinstance(obj, str):
        try:
            return datetime.fromisoformat(obj)
        except ValueError:
            return obj
    else:
        return obj

 # Function to recursively convert "true"/"false" strings to Booleans
def convert_bools(obj):
    if isinstance(obj, dict):
        return {k: convert_bools(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_bools(elem) for elem in obj]
    elif obj == "true":
        return True
    elif obj == "false":
        return False
    else:
        return obj

In [16]:
# get performance
async def getPerformance(type):
    performance = await send_get_request('http://participant0.local:5000/api/performance',type='json')

    filteredPerformance = {}
    for k,v in performance.items():
        if v['event']==type:
            filteredPerformance[k] = v
    #filter performance by type
    return filteredPerformance
await getPerformance('dlrp')

DEBUG    [connectionpool.py:241] Starting new HTTP connection (1): participant0.local:5000
DEBUG    [connectionpool.py:544] http://participant0.local:5000 "GET /api/performance HTTP/1.1" 200 990


{'2025-08-02T12:00:00': {'baselineW': [108.2326114814815,
   154.50829939814815,
   236.75033296296297,
   134.5920723611111],
  'button': {'offPause': [datetime.datetime(2025, 8, 3, 15, 44, 19, 464179)],
   'onPause': []},
  'datetime': datetime.datetime(2025, 8, 3, 12, 0),
  'event': 'dlrp',
  'flexW_avg': 145.48615717592594,
  'goalAvg': 0.8992456181866284,
  'goalPerc': [0.8342565490915176,
   1.0,
   0.9778321272696118,
   0.7848937963853845],
  'loadW_avg': 13.034671875,
  'loadW_hourly': [17.938846527777777,
   0.0,
   5.248251250000001,
   28.951589722222224],
  'modified': datetime.datetime(2025, 8, 3, 15, 44, 22, 844849)},
 '2025-08-03T15:00:00': {'baselineW': [100.94405427083332,
   109.37338708333334,
   145.98361538194445,
   103.36052958333335],
  'button': {'offPause': [], 'onPause': []},
  'datetime': datetime.datetime(2025, 8, 3, 15, 0),
  'event': 'dlrp',
  'flexW': [64.1509882986111, 109.37338708333334, 145.98361538194445, 0],
  'flexW_avg': 79.87699769097222,
  'goa

{'2025-08-02T12:00:00': {'baselineW': [108.2326114814815,
   154.50829939814815,
   236.75033296296297,
   134.5920723611111],
  'button': {'offPause': [datetime.datetime(2025, 8, 3, 15, 44, 19, 464179)],
   'onPause': []},
  'datetime': datetime.datetime(2025, 8, 3, 12, 0),
  'event': 'dlrp',
  'flexW_avg': 145.48615717592594,
  'goalAvg': 0.8992456181866284,
  'goalPerc': [0.8342565490915176,
   1.0,
   0.9778321272696118,
   0.7848937963853845],
  'loadW_avg': 13.034671875,
  'loadW_hourly': [17.938846527777777,
   0.0,
   5.248251250000001,
   28.951589722222224],
  'modified': datetime.datetime(2025, 8, 3, 15, 44, 22, 844849)},
 '2025-08-03T15:00:00': {'baselineW': [100.94405427083332,
   109.37338708333334,
   145.98361538194445,
   103.36052958333335],
  'button': {'offPause': [], 'onPause': []},
  'datetime': datetime.datetime(2025, 8, 3, 15, 0),
  'event': 'dlrp',
  'flexW': [64.1509882986111, 109.37338708333334, 145.98361538194445, 0],
  'flexW_avg': 79.87699769097222,
  'goa